<a href="https://colab.research.google.com/github/Git2723122/y4-dissertation/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.3 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((x_train.shape[0], 28, 28 ,1))
x_test = x_test.reshape((x_test.shape[0], 28, 28 ,1))

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
def build_model(hp):
  model = Sequential([
      Conv2D(filters=hp.Int('filters1', 16, 64, step=16), kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
      MaxPooling2D((2,2)),
      Dropout(rate=hp.Float('rate1',0.1, 0.5, step=0.1)),

      Conv2D(filters=hp.Int('filters2', 32, 128, step=32), kernel_size=(3,3), activation='relu',input_shape=(28,28,1)),
      MaxPooling2D((2,2)),
      Dropout(rate=hp.Float('rate2',0.1,0.5,step=0.1)),

      Flatten(),
      Dense(units=hp.Int('units',64,256, step=64), activation='relu'),
      Dropout(rate=hp.Float('rate3',0.1,0.5,step=0.1)),

      Dense(10,activation='softmax')
  ])
  model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuning',
    project_name='tuning'
)

tuner.search(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

tuned_hps = tuner.get_best_hyperparameters(num_trials=3)[0]
print(f"Best Hyperparameters: {tuned_hps.values}")
tuned_model = tuner.hypermodel.build(tuned_hps)
tuned_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Input(shape=(28,28,1)))
  model.add(Conv2D(filters=hp.Int('filters_base', 16, 64, step=16), kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(rate=hp.Float('rate_base', 0.1, 0.5, step=0.1)))

  for i in range(hp.Int('num_layers', 1, 3)):
    model.add(Conv2D(filters=hp.Int(f'filters_{i}', 32, 128, step=32), kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(rate=hp.Float(f'rate_{i}', 0.1, 0.5, step=0.1)))

  model.add(Flatten())
  model.add(Dense(units=hp.Int('units', 64, 256, step=64), activation='relu'))
  model.add(Dropout(rate=hp.Float('rate_dense', 0.1, 0.5, step=0.1)))

  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

  return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuning',
    project_name='tuning'
)

tuner.search(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

tuned_hps = tuner.get_best_hyperparameters(num_trials=3)[0]
print(f"Best Hyperparameters: {tuned_hps.values}")
tuned_model = tuner.hypermodel.build(tuned_hps)
tuned_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 01s]

Best val_accuracy So Far: 0.9610000252723694
Total elapsed time: 00h 03m 19s
Best Hyperparameters: {'filters_base': 16, 'rate_base': 0.30000000000000004, 'num_layers': 2, 'filters_0': 64, 'rate_0': 0.4, 'units': 192, 'rate_dense': 0.30000000000000004, 'learning_rate': 0.001, 'filters_1': 64, 'rate_1': 0.5, 'filters_2': 32}
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 48s 24ms/step - accuracy: 0.6232 - loss: 1.0878 - val_accuracy: 0.9630 - val_loss: 0.1296
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 24ms/step - accuracy: 0.9064 - loss: 0.3147 - val_accuracy: 0.9745 - val_loss: 0.0884


In [6]:
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train_flat)
x_test_scaled = scaler.transform(x_test_flat)

## Reduce training size TESTING ONLY

In [ ]:
x_train_scaled = x_train_scaled[:10000]
y_train = y_train[:10000]

In [ ]:
clf = SVC(kernel='rbf', C=1.0, gamma='scale')
clf.fit(x_train_scaled, y_train.argmax(axis=1))

In [6]:
predsvm = clf.predict(x_test_scaled)
print(classification_report(y_test.argmax(axis=1), predsvm))


              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.97      0.96      1032
           3       0.97      0.97      0.97      1010
           4       0.97      0.96      0.97       982
           5       0.96      0.96      0.96       892
           6       0.98      0.97      0.98       958
           7       0.93      0.96      0.94      1028
           8       0.96      0.95      0.96       974
           9       0.97      0.94      0.95      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000



In [ ]:
lrclf = LogisticRegression(solver='saga', max_iter=1000, verbose=1)
lrclf.fit(x_train_scaled, y_train.argmax(axis=1))

In [8]:
predlr = lrclf.predict(x_test_scaled)
print(classification_report(y_test.argmax(axis=1),predlr))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       980
           1       0.96      0.98      0.97      1135
           2       0.92      0.89      0.90      1032
           3       0.90      0.91      0.91      1010
           4       0.94      0.93      0.93       982
           5       0.89      0.87      0.88       892
           6       0.94      0.95      0.95       958
           7       0.93      0.92      0.92      1028
           8       0.87      0.88      0.88       974
           9       0.91      0.92      0.91      1009

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



In [7]:
knnclf = KNeighborsClassifier(n_jobs=-1)
knnclf.fit(x_train_scaled, y_train.argmax(axis=1))

KNeighborsClassifier(n_jobs=-1)

In [8]:
predknn = knnclf.predict(x_test_scaled)
print(classification_report(y_test.argmax(axis=1), predknn))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       980
           1       0.96      0.99      0.97      1135
           2       0.96      0.93      0.94      1032
           3       0.92      0.95      0.94      1010
           4       0.94      0.94      0.94       982
           5       0.93      0.92      0.93       892
           6       0.96      0.97      0.97       958
           7       0.94      0.92      0.93      1028
           8       0.96      0.90      0.93       974
           9       0.92      0.92      0.92      1009

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000

